# Polymorphic types in the lambda notebook

Updated 2/23/24

The lambda notebook supports polymorphic types (variables over types) in certain cases.  Specifically, what is implemented is a version of the Hindley-Milner type system.

## Introduction

**To a first approximation**: _a type variable stands in for "any type"_.

Of course, in order to keep type inference computable, it cannot actually be any type, but the limitations will seldom matter for linguistic purposes. The power of type variables comes when putting them into complex types.

### Notation

Mechanically, you can write type variables using `X`, `Y`, and `Z` followed by any number or number of primes.  So for example, `<X,Y15>` is a function from some type `X` to some type `Y15`.  You also may occasionally see some variable types prefixed with the symbol `?`, which emerge under certain conditions from internal type computations in order to avoid variable name collisions.

### The set interpretation for type variables

One interpretation of types that contains variables is as sets of types. So for example, the type `<X,Y>` describes the set of all concrete functional types, for example `<e,t>`, `<e,e>`, `<e,<e,t>>`, `<<e,e>,t>`, etc., but *not* just `e` or `t`. A type like `<X,t>` would therefore describe a subset of `<X,Y>`: all those functional types that end in `t`. Similarly, a type like `<X,X>` describes a different subset of of `<X,Y>`: the set of all functional types whose input and output type are the same. Out of context, type variable names don't matter. That is, types `X`, `Y`, and other alphabetic variants with the same structure characterize the same sets. Similarly for `<X,X>`, `<Y,Y>`, and so on. Of course *in* context variable names do matter. We have already seen that `<X,Y>` and `<X,X>` do not characterize the same sets.

On the set interpretation, type inference involves the question of: given two types, is there a common subset, holding type variables fixed between the types? For example, `<e,X>` describes the set of functional types with concrete input type `e`, and `<X,e>` the set of functional types with concrete return type `e`. The common subset for these two types is just the set containing `<e,e>`.

A weird case: is there a common subset for types `X` and `<X,X>`? On this type system the answer is a definitive **no**, and the set interpretation helps see why. The right type characterizes the set of functions whose inputs and outputs would both be arbitrary type `X`. To resolve this with plain type `X`, we would have to suppose that the set characterized by `<X,X>` is a subset of the set characterized by `X`, and therefore we have functional types that need non-finite type expressions to characterize (flavors of a Russell-like paradox). The Hindley-Milner system rules this out. This case is conventionally called an *occurs check* failure (where plain `X` on one side occurs recursively on the other side).

### Interpretation via unification

The set-based interpretation above is not standard in computer science literature on type theory, as helpful as it may be for those from linguistics/philosophy training. Rather, type inference is usually couched in terms of *variable unification*. Given some sequence of formulas with variables, can we find a consistent and coherent mutual resolution of the variables? This is of course a hard and potentially intractable problem in the general case, but type inference of the kind that concerns us here is essentially first-order unification, which is tractable and well-understood. The lambda notebook implementation and api uses the unification interpretation, so it is useful to see.

Terminology: given two types, if unification succeeds it returns a unique (see caveat below) **principal type** that serves as the unifier between the two.

For example, consider the problem of unifying types `<e,X>` and `<Y,t>`. We are looking for an assignment to `X` and `Y` that gives some valid type. In this case, the assignment is straightforward: `X=t, Y=e`:

In [1]:
types.poly_system.unify_details(tp("<e,X>"), tp("<Y,t>"))

Principal type:,"$\left\langle{}e,t\right\rangle{}$"
Inputs:,"$\left\langle{}e,X\right\rangle{}$ and $\left\langle{}Y,t\right\rangle{}$"
Mapping:,"{$Y$: $e$, $X$: $t$}"


As another example, consider the types `<e,X>` and `<t,X>`. In this case, there is no assignment to `X` that can resolve the mismatch of `e` and `t`.

In [2]:
# n.b. this function by default returns `None` for this case, and the named argument
# is needed to actually show some output
types.poly_system.unify_details(tp("<e,X>"), tp("<t,X>"), show_failure=True)

Principal type:,Unification failure!
Inputs:,"$\left\langle{}e,X\right\rangle{}$ and $\left\langle{}t,X\right\rangle{}$"


This unification failure is straightforward, but other cases become less so, e.g. `<e,<X,X>>` and `<Y,<Y,t>>` fail for essentially the same reason. `X` must be equal to `Y`, by looking at the return type's input type. But from main input type, we infer that `Y=e`, and therefore `X=e` (assignments are interpreted transitively). However, we also most infer from the ultimate return types that `X=t`, and therefore have arrived at a contradiction.

This algorithm works with fully non-concrete types as well. Consider `<X,Y>` and `<X1,<X,X1>>`. This is a bit trickier, but it is also resolvable: an initial assignment `X=X1, Y=<X,X1>` will do the trick; we need to take the transitive substitution of this mapping, which then gives `Y=<X1,X1>` and therefore principal type `<X1,<X1,X1>`. From this example you can see there is not guaranteed to be a unique assignment for variable type unification; the initial assignment `X1=X, Y=<X,X1>` will also do the trick, giving principal type `<X,<X,X>>`. These two types are not string-identical, but in this type system they have the same interpretation. So unification is, more generally stated, guaranteed to return a principal type that is unique up to alpha variants. Unification is symmetric, but again only up to equivalence under alpha variants.

In [3]:
types.poly_system.unify_details(tp("<X,Y>"), tp("<X1,<X,X1>>"))

Principal type:,"$\left\langle{}X',\left\langle{}X',X'\right\rangle{}\right\rangle{}$"
Inputs:,"$\left\langle{}X,Y\right\rangle{}$ and $\left\langle{}X',\left\langle{}X,X'\right\rangle{}\right\rangle{}$"
Mapping:,"{$X$: $X'$, $Y$: $\left\langle{}X',X'\right\rangle{}$}"


In [4]:
types.poly_system.unify_details(tp("<X1,<X,X1>>"), tp("<X,Y>"))

Principal type:,"$\left\langle{}X,\left\langle{}X,X\right\rangle{}\right\rangle{}$"
Inputs:,"$\left\langle{}X',\left\langle{}X,X'\right\rangle{}\right\rangle{}$ and $\left\langle{}X,Y\right\rangle{}$"
Mapping:,"{$X'$: $X$, $Y$: $\left\langle{}X,X\right\rangle{}$}"


*Occurs check failures redux*: An occurs check case like `X` + `<X,X>` would result in an immediate assignment like `X=<X,X>`. However, when taking the closure of this assignment under substitution, we would need a non-finite type for `X` (replacements `X=<<X,X>,<X,X>>`, `X=<<<X,X>,<X,X>>,<<X,X>,<X,X>>>`, ...). This is disallowed.

Technical note: in internal unify calls, including `unify_details` that is used above, occurs check failures raise an exception (`types.OccursCheckFailure`, a subclass of `TypeMismatch`) rather than returning `None`.

In [5]:
with lamb.errors():
    types.poly_system.unify_details(tp("X"), tp("<X,X>"))

<span style="color:red">**TypeMismatch**</span>: type $X$ and type $\left\langle{}X,X\right\rangle{}$ conflict (Occurs check failed)

**Internals**. This document won't explain how to actually do unification (i.e. find a coherent assignment iff one exists in the system). The algorithm that the lambda notebook uses for doing type unification is sometimes called _Algorithm U_, after Damas and Milner (1982), _Principal type-schemes for functional programs_, proceedings of ACM POPL, and is similar in the abstract to what is used in many functional programming languages. The lambda notebook implementation should be thought of as a research implementation aimed only at linguistics, and it isn't optimized for doing programming per se. (If you want that, consider Haskell.)

The algorithm for doing type inference in the metalanguage is an implementation of Damas and Milner's _Algorithm W_.  This amounts to unifying the constraints imposed by metalanguage combination across an entire formula, rather than just a single type.  For example, in the generalized PM example below, we can infer both that `X` maps to `e` and that this is consistent across the formula.  In practice, most cases that are relevant to formal semantics are about this simple.

## Example: Generalized predicate modification

In practice, much of formal semantics does not require variable types.  One kind of use in the lambda notebook (and elsewhere) is writing general-purpose combinators.  For example, consider the combinator for the standard Heim and Kratzer implementation of Predicate Modification:

In [6]:
%te L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)

(λ f_<e,t>: (λ g_<e,t>: (λ x_e: (f_<e,t>(x_e) & g_<e,t>(x_e)))))

Suppose you wanted to generalize this to other types, e.g. events, worlds, or perhaps some higher types. It's easy to write new combinators at those types, but this doesn't capture the common structure across them. To capture this structure, you can use type variables. Here is a combinator for doing generalized _Predicate Modification_ over arbitrary property types:

In [7]:
%%lamb
gpm = L f_<X,t> : L g_<X,t> : L x_X : f(x) & g(x)

${gpm}_{\left\langle{}\left\langle{}X,t\right\rangle{},\left\langle{}\left\langle{}X,t\right\rangle{},\left\langle{}X,t\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{\left\langle{}X,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}X,t\right\rangle{}} \: . \: \lambda{} x_{X} \: . \: {f}({x}) \wedge{} {g}({x})$

The type variable here signals that the function gpm is underspecified for some type `X`.  To resolve the variables, all of the `X`s in this formula must be consistent, so whatever `X` is, `g` and `f` must have the same type.  We can see this by combining `gpm` with an argument:

In [8]:
gpm(te("L x_e : Cat_<e,t>(x)")).reduce_all()

(λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e))))

In [9]:
gpm(te("L x_e : Cat_<e,t>(x)")).reduce_all().derivation.trace()

DisplayNode instance: HTML rendering only

What has happened here?  The type system inferred at the initial combination stage (reduction is not required) that `X` must be equal to `e`, a non-variable type, and systematically replaced the type variable across all instances in the function. Of course, this works with any value for `X`:

In [10]:
gpm(te("L x_n : Even_<n,t>(x)"))

(λ f_<n,t>: (λ g_<n,t>: (λ x_n: (f_<n,t>(x_n) & g_<n,t>(x_n)))))(λ x_n: Even_<n,t>(x_n))

Type-flexible combinators are an extremely powerful tool for expressing composition operations. For further examples in the provided documentation, see the Variable free fragment and the Continuations and Quantifier scope fragment. The module `lamb.combinators` provides a few standard combinators, for example, the z combinator that is familiar from categorial grammar:

In [11]:
lamb.combinators.z_combinator

(λ f_<X,<Y,Z>>: (λ g_<Y,X>: (λ x_Y: f_<X,<Y,Z>>(g_<Y,X>(x_Y))(x_Y))))

The Geach combinator is discussed as an example below.

## Example: function-argument type inference

A somewhat more sophisticated example is the implementation of type-checking for function-argument combination in the metalanguage.  The most straightforward algorithm you might imagine would go something like:

 1. Is `fun.type` a functional type?  If not, fail.
 2. Does `fun.type.left` equal `arg.type`?  If not, fail.
 3. Otherwise, succeed.
 
This is a perfectly fine procedure for concrete types.  If variable types are in the mix, we can do something a bit more interesting, and in fact need to:

 1. Does `fun.type` match `<arg.type,X>`, where `X` is new?  If not, fail.
 2. Otherwise, succeed.

Here is an example of this unification step for the straightforward case of function type `<e,t>` and argument type `e`:

In [12]:
types.poly_system.unify(tp("<e,t>"), tp("<e,X>"))

<e,t>

Even more generally: given some arbitrary potential function $f$, and potential argument $x$, does unifying $f.type$ with $\langle x.type,X\rangle$ (for unused `X`) succeed? If so, adjust $f$ and $x$'s type accordingly. Another example that is useful to consider is the basic function-argument combination of two variable types.  The function's variable gets mapped into a functional type over variables, where the first one matches the argument's variable type. Here is this most general case:

In [13]:
%te f_X(a_Y)

f_<Y,X>(a_Y)

Note that the `X` in the output is not assumed to be the same `X` as in the input.

Further inference is of course possible in context, for example, binary truth-functional operators would force `X` in the above result to get mapped to `t`.

In [14]:
%te f_X(a_Y) & p_t

(f_<Y,t>(a_Y) ∧ p_t)

## Example: the Geach combinator

Here is another example, the famous _Geach rule_ in combinator form (what, for example, Jacobson calls the _g-combinator_.

In [26]:
%%lamb
geach = L g_<Y,Z> : L f_<X,Y> : L x_X : g(f(x))

${geach}_{\left\langle{}\left\langle{}Y,Z\right\rangle{},\left\langle{}\left\langle{}X,Y\right\rangle{},\left\langle{}X,Z\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} g_{\left\langle{}Y,Z\right\rangle{}} \: . \: \lambda{} f_{\left\langle{}X,Y\right\rangle{}} \: . \: \lambda{} x_{X} \: . \: {g}({f}({x}))$

This combinator generalizes function composition, for functions of arbitrary types.  `gc(g)(f)(x)` is equivalent to `(g * f)(x)` where `*` is function composition.  Since the metalanguage allows for `*` to signal function composition, we can see this directly:

In [29]:
%%lamb
f = L x_n : x+1
g = L x_n : x>3

${f}_{\left\langle{}n,n\right\rangle{}}\:=\:\lambda{} x_{n} \: . \: \textsf{1} + {x}$<br />
${g}_{\left\langle{}n,t\right\rangle{}}\:=\:\lambda{} x_{n} \: . \: \textsf{3} < {x}$

In [30]:
g * f

(λ x_n: (3_n < (1_n + x_n)))

In [31]:
geach(g)(f).reduce_all()

(λ x_n: (3_n < (1_n + x_n)))

In [32]:
geach(g)(f).reduce_all().derivation

1. (λ g_<n,t>: (λ f_<X,n>: (λ x_X: g_<n,t>(f_<X,n>(x_X)))))(λ x_n: (3_n < x_n))(λ x_n: (1_n + x_n))    (Type inference)
 2. (λ g_<n,t>: (λ f_<n,n>: (λ x_n: g_<n,t>(f_<n,n>(x_n)))))(λ x_n: (3_n < x_n))(λ x_n: (1_n + x_n))    (recursive reduction of subexpression 0)
 3. (λ f_<n,n>: (λ x_n: (3_n < f_<n,n>(x_n))))(λ x_n: (1_n + x_n))    (Reduction)
 4. (λ x_n: (3_n < (λ x_n: (1_n + x_n))(x_n)))    ([[Reduction]])
 5. (λ x_n: (3_n < (1_n + x_n)))

In [33]:
%lamb h = L p_t : p & Q_t

${h}_{\left\langle{}t,t\right\rangle{}}\:=\:\lambda{} p_{t} \: . \: {Q}_{t} \wedge{} {p}$

In [34]:
h * g

(λ x_n: (Q_t & (3_n < x_n)))

In [35]:
geach(h).derivation

1. (λ g_<Y,Z>: (λ f_<X,Y>: (λ x_X: g_<Y,Z>(f_<X,Y>(x_X)))))(λ p_t: (Q_t & p_t))    (Type inference)
 2. (λ g_<t,t>: (λ f_<X,t>: (λ x_X: g_<t,t>(f_<X,t>(x_X)))))(λ p_t: (Q_t & p_t))

In [36]:
geach(h)(g).derivation

1. (λ g_<t,t>: (λ f_<X,t>: (λ x_X: g_<t,t>(f_<X,t>(x_X)))))(λ p_t: (Q_t & p_t))(λ x_n: (3_n < x_n))    (Type inference)
 2. (λ g_<t,t>: (λ f_<n,t>: (λ x_n: g_<t,t>(f_<n,t>(x_n)))))(λ p_t: (Q_t & p_t))(λ x_n: (3_n < x_n))

In [37]:
geach(h)(g).reduce_all().derivation.trace()

DisplayNode instance: HTML rendering only

## Type variable binding and internal types (advanced)

In this type system, type variables are implicitly interpreted as *universally bound*. That is, when we right something like `<X,Y>`, we are implicitly considering "the type such that for any `X` and `Y` is described by `<X,Y>`. For most practical cases, you can think of the scope of this binding as "global relative to a metalanguage expression". That is, when writing an expression within a `%te` magic, the type variables have the same meaning at any point in that formula, but variables are bound at the top of that formula only. So an `X` in two distinct `%te` definitions (or lines in a `%%lamb` magic) is not expected to be the "same" `X`. The type system does not support any other sorts of binders, but as an advanced technique, it does support certain manipulations of variable scoping. There are two mechanisms for this, one at the metalanguage level and one at the type level.

### Let binding

The metalanguage supports what in Hindley-Milner-based systems is typically termed `let` operators, though in quite restricted form.  A `let` operator binds type variables and treats them as _universal_/_generic_.  A variable inside a `let` operator does not have to obey constraints imposed on a variable of the same name not in the scope of that `let`.  Let-scoped expressions may therefore safely have their variables renamed as long as internal type constraints are obeyed.

For purposes of formal semantics, one does not typically have to pay much attention to let-scoping or mess with it (in direct contrast to functional programming languages), and so the metalanguage does not indicate it or allow for much manipulation of it.  It is possible to disable let in direct calls to `te`, but currently I don't see that this should be much needed.  Here is the above function-argument example without a let operator.

In [15]:
te("f_X(a_Y)", let=False)

f_<Y,?27>(a_Y)

The resulting type includes a `?` type, which corresponds to a new type variable (sometimes termed a "fresh" type) variable used during type inference. A `let` operator would indicate that it is safe to rename this variable for the sake of a human readability; we can explicitly see this from "compacting" the type variables:

In [16]:
te("f_X(a_Y)", let=False).compact_type_vars()

f_<Y,X>(a_Y)

For most purposes you will never have to see this, but it can be useful to be aware that at the type inference level, function-argument inference is implemented using fresh variables. The numbers on fresh variables may change arbitrarily from call to call.

In [18]:
types.poly_system.unify_fa(tp("Z"), tp("Y"))

(<Y,?32>, Y, ?32)

In programming languages (/programming language theory) a typical syntax for the let operator is something like:

    let f = lambda x: x
    in
        f(y)
    end

The type of `x` in this sort of syntax would be treated as generic relative to any type constraints imposed or inferred in the body.  This syntax is implicit in the `%%lamb` magic: any assignment to a variable or lexical item is treated as in the definitional part of a `let` statement, and any use later of that variable/item is treated as having generic type variables.

One caveat about this: since all `te`-type instantiations of metalanguage formulas amount to this sort of `let` context, if you reuse variable names, you can get somewhat counterintuitive differences between direct combinations of formulas and indirect ones. For example, the following is well-formed and has a valid type; the instances of `X` are distinct from each other.

In [19]:
te("L x_X : x")(te("L x_X : x"))

(λ x_<X,X>: x_<X,X>)(λ x_X: x_X)

However, the following is not: type variable inference fails because the two `X`s are the same, and unifying them would lead to recursion, and therefore an occurs check failure:

In [21]:
%te (L x_X : x)(L x_X : x)

<span style="color:red">**TypeMismatch**</span>: `(λ x_X: x_X)/<X,X>` and `(λ x_X: x_X)/X` conflict (Occurs check failure while trying to infer function type given argument `<X,X>`)

**As a limiting case:** All type variables are interpreted as "globally" let-bound.

### $\forall$ types

The type system itself allows limited explicit type variable binding via the unselective `∀` operator. This can be useful for "type hinting" without reference to surrounding variable types. Essentially, a type like `∀<X,<Y,X>>` indicates that `X` and `Y` should be compared for identity only within the scope of the `∀` binder. This can be useful when writing a complex formula.

In [22]:
%te L q_X : p_t & f_∀<Y,Y>(q)

(λ q_t: (p_t & f_<t,t>(q_t)))

As a convenience, the type written as a simple `?` will translate as `∀X`, to give a guaranteed fresh variable without using internal types.

In [25]:
%te f_?(y_?)

f_<∀X,X>(y_∀X)

As a reminder, let-bound type variables are also interpreted as universally bound. So a let-bound type $X$ is implicitly equivalent to `∀X`, but for the sake of simplicity the $\forall$ is not present.

One subtlety is this: unification is constrained to preserve variable identity for "free" variables, because they might be let-bound in a context it can't see. With this constraint, it is possible to unify certain cases of `∀` types that do not have a bound expressable output. The type inference system for this case will output internal `?` types, under the assumption that these types are going to be let-bound somewhere. For example, consider the types `<∀X,<Y,∀X>>` and `∀<X,<Y,X>>`. The identity of `Y` in the left type needs to be preserved, and the unification of the `∀`-bound `Y` in the right type and the `Y` in the left type is a non-locally-bound `Y`. At the same time, all of the outer `X`s are `∀` bound, and so ideally would be `∀` bound in the output as well. There is no unselective binding of the resulting type, which with selective binding might come out as something like "`∀X : <X, <Y, X>>`". Unification therefore will fall back on `?` types for the X variables.

In [39]:
types.poly_system.unify(tp('<∀X,<Y,∀X>>'), tp('∀<X,<Y,X>>'))

<?82,<Y,?82>>

Of course, in the context of a full metalanguage expression, let-binding will capture these variables, resulting in a more nicely behaved result:

In [45]:
%te f_<∀X,<Y,∀X>> <=> f_∀<X,<Y,X>>

(f_<X,<Y,X>> <=> f_<X,<Y,X>>)

In [44]:
%te simplify f_<∀X,<Y,∀X>> <=> f_∀<X,<Y,X>>

True_t